In [ ]:
# ID: 2018115809 (undergraduate)
# NAME: Dohun Kim
# File name: synopsis_vector.ipynb
# Platform: Python 3.9.5 on Ubuntu Linux 18.04
# Required Package(s): numpy=1.20.3, pandas=1.2.4

# Synopsis-to-Vector

In [1]:
import numpy as np
import pandas as pd
import pickle
import math

## Load Pre-trained CBOW Parameters

In [2]:
!wget -nc -q https://github.com/WegraLee/deep-learning-from-scratch-2/raw/master/ch04/cbow_params.pkl

In [3]:
pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

## Get TF-IDF from synopsis.csv

### Load Cleaned Synopsis Data

In [4]:
synopsis = pd.read_csv('../clean_data/synopsis.csv')
synopsis.head()

,MAL_ID,Synopsis
0,1,solar leaving surface planet earth solar polic...
1,5,day life crew routine interrupted chasing targ...
2,6,head reason waste oppose entire cities fun tit...
3,7,individuals powers mind control robin craft us...
4,8,dark century people suffering rule manipulate ...


### Get Word Set from Synopsis Data

In [5]:
syns = synopsis.Synopsis.str.findall('\w+')

vocab = set()
for syn in syns:
    vocab.update(syn)

vocab = sorted(list(vocab))
vocab_size = len(vocab)

### Get TF(Term Frequency)

In [6]:
def get_tf(word, syn):
    # Term Frequency
    return syn.count(word)

    
tf = []
for syn in syns:
    tf_syn = []
    for word in vocab:
        tf_syn.append(get_tf(word, syn))
    tf.append(tf_syn)

tf = np.asarray(tf)
tf.shape

(6000, 5918)

### Get IDF(Inverse Document Frequency)

In [7]:
def get_idf(term):
    # Inverse Document Frequency
    _df = 0
    for syn in syns:
        _df += int(word in syn)
    return math.log(vocab_size/(_df))


idf = []
for word in vocab:
    idf.append(get_idf(word))

idf = np.asarray(idf)
idf.shape

(5918,)

### Get TF-IDF Table

In [8]:
tf_idf = pd.DataFrame(tf * idf, columns=vocab)

## Generate Sentence Vector of Synopsis

In [9]:
def get_word_vec(word):
    return word_vecs[word_to_id[word]]

def get_tf_idf(word, i):
    return tf_idf.loc[i, word]

def get_sent_vec(i):
    sent_vec = np.zeros(100, dtype=np.float16)

    for word in set(syns[i]):
        sent_vec += get_word_vec(word) * get_tf_idf(word, i)
        # sent_vec += get_word_vec(word)
    
    return sent_vec / len(set(syns[i]))

In [10]:
full_sent_vec = []
for i in range(len(syns)):
    full_sent_vec.append(get_sent_vec(i))
    
sent_vec_df = pd.DataFrame(full_sent_vec)
sent_vec_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.115479,-3.919922,0.511230,1.325195,2.685547,1.791992,-1.767578,-1.251953,-1.898438,1.073242,...,0.862305,-0.740234,-1.121094,-0.729004,1.063477,1.272461,-0.246826,1.380859,1.921875,2.216797
1,2.314453,-2.361328,-0.012909,2.703125,1.200195,3.173828,-1.356445,-2.605469,-1.958984,-0.565918,...,0.493652,-0.944824,-0.937500,-0.261475,2.087891,1.500000,-0.221802,0.646973,2.218750,0.958496
2,-0.226196,-2.597656,-0.182983,2.042969,1.670898,1.951172,-1.367188,-2.917969,-0.160767,0.050781,...,-1.435547,-1.021484,-2.537109,-2.455078,2.013672,1.943359,0.622559,1.703125,2.357422,1.232422
3,0.844238,-0.527344,0.271973,0.752930,1.575195,2.492188,-1.652344,-2.705078,-2.269531,1.469727,...,-0.367920,-1.791992,-1.804688,-2.474609,2.878906,0.425537,-0.583008,3.107422,0.836426,1.042969
4,1.521484,-2.650391,0.424072,1.816406,2.029297,1.694336,0.607910,-1.333984,-1.023438,0.936035,...,1.650391,-1.703125,-2.476562,-4.492188,1.403320,0.638672,0.058594,1.249023,2.457031,2.517578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.906250,-1.420898,1.903320,2.382812,1.667969,0.900391,-1.924805,-0.842285,-1.282227,1.339844,...,1.071289,-0.566406,-1.356445,-2.287109,0.418701,1.645508,0.828613,1.502930,2.164062,2.941406
5996,2.138672,-1.449219,0.350586,2.123047,0.429688,-0.171265,-0.983398,-2.195312,-2.029297,1.023438,...,-0.836914,-1.897461,-2.449219,-2.023438,1.227539,0.563477,-1.253906,2.990234,2.492188,1.022461
5997,1.149414,-2.382812,0.937500,0.431152,1.120117,2.523438,-1.699219,-1.219727,-1.711914,1.555664,...,0.713379,-0.082703,-1.974609,-0.666504,2.468750,0.974609,1.821289,-0.183594,2.945312,0.713867
5998,0.945801,-1.449219,-0.097290,0.824707,1.160156,1.100586,-0.374756,-2.130859,-0.812012,2.248047,...,1.913086,0.848145,-1.114258,-1.065430,1.032227,1.029297,-0.781738,0.201904,2.699219,1.934570


In [11]:
sent_vec_df.insert(0, 'MAL_ID', synopsis.MAL_ID)
sent_vec_df

,MAL_ID,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,1,-0.115479,-3.919922,0.511230,1.325195,2.685547,1.791992,-1.767578,-1.251953,-1.898438,...,0.862305,-0.740234,-1.121094,-0.729004,1.063477,1.272461,-0.246826,1.380859,1.921875,2.216797
1,5,2.314453,-2.361328,-0.012909,2.703125,1.200195,3.173828,-1.356445,-2.605469,-1.958984,...,0.493652,-0.944824,-0.937500,-0.261475,2.087891,1.500000,-0.221802,0.646973,2.218750,0.958496
2,6,-0.226196,-2.597656,-0.182983,2.042969,1.670898,1.951172,-1.367188,-2.917969,-0.160767,...,-1.435547,-1.021484,-2.537109,-2.455078,2.013672,1.943359,0.622559,1.703125,2.357422,1.232422
3,7,0.844238,-0.527344,0.271973,0.752930,1.575195,2.492188,-1.652344,-2.705078,-2.269531,...,-0.367920,-1.791992,-1.804688,-2.474609,2.878906,0.425537,-0.583008,3.107422,0.836426,1.042969
4,8,1.521484,-2.650391,0.424072,1.816406,2.029297,1.694336,0.607910,-1.333984,-1.023438,...,1.650391,-1.703125,-2.476562,-4.492188,1.403320,0.638672,0.058594,1.249023,2.457031,2.517578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,48438,0.906250,-1.420898,1.903320,2.382812,1.667969,0.900391,-1.924805,-0.842285,-1.282227,...,1.071289,-0.566406,-1.356445,-2.287109,0.418701,1.645508,0.828613,1.502930,2.164062,2.941406
5996,48466,2.138672,-1.449219,0.350586,2.123047,0.429688,-0.171265,-0.983398,-2.195312,-2.029297,...,-0.836914,-1.897461,-2.449219,-2.023438,1.227539,0.563477,-1.253906,2.990234,2.492188,1.022461
5997,48470,1.149414,-2.382812,0.937500,0.431152,1.120117,2.523438,-1.699219,-1.219727,-1.711914,...,0.713379,-0.082703,-1.974609,-0.666504,2.468750,0.974609,1.821289,-0.183594,2.945312,0.713867
5998,48483,0.945801,-1.449219,-0.097290,0.824707,1.160156,1.100586,-0.374756,-2.130859,-0.812012,...,1.913086,0.848145,-1.114258,-1.065430,1.032227,1.029297,-0.781738,0.201904,2.699219,1.934570


In [12]:
sent_vec_df.to_csv('synopsis_vector.csv', index=False)